# Test Matching Insee/ETS

## INSEE

- https://s3.console.aws.amazon.com/s3/object/calfdata/INSEE/Stock/ETS/
        - INSEE/Stock/ETS/StockEtablissement_utf8.csv
        
```
['siren', 'siret']
```

## INPI

- https://s3.console.aws.amazon.com/s3/buckets/calfdata/INPI/TC_1/Stock_processed/
    - INPI/TC_1/Stock_processed/initial_ETS.gz
    - INPI/TC_1/Stock_processed/initial_ETS.json
    
Colonnes test:

```
["Siren","Date_Immatriculation", "Date_Clôture", "Date_Greffe"]
```

## Sauvegarde

* La liste des SIREN matchés sera sauvegardée selon leur nature et origine
  * nature → ACTES/COMPTES/ETS/etc
  * origine → initial/partiel/new/evt

Les matchés seront sauvegardé dans calfdata/SIRETISATION/matche/ au format suivant:

* insee_nature_origine_matche.gz
    * ex: insee_pm_initial_matche.gz
    
    

## Moteur de recherche TEST

* Insee
  * http://avis-situation-sirene.insee.fr/IdentificationListeSiret.action
* INPI/TC
  * https://data.inpi.fr/
* Infogreffe
  * https://www.infogreffe.fr/


In [1]:
import boto3, json
import dask.dataframe as dd
import pandas as pd
import Match_inpi_insee.aws_connectors as aws
#from tqdm.notebook import tqdm
#import tqdm
%load_ext autoreload
%autoreload 2

In [2]:
instance_aws = 'https://calfdata.s3.eu-west-3.amazonaws.com'
bucket = 'calfdata'

In [3]:
# instanciate AWS connection
AWS_connection = aws.aws_instantiate(instance_aws, bucket)

## Valeurs communes dans INSEE/INPI ETS

| INSEE                     | INPI         |                         |
|---------------------------|--------------|-------------------------|
| Var                       | Var          | comment                 |
| siren                     | Siren        |                         |
| codePostalEtablissement   | Code_Postal  | Si Type == 'principale' |
| codePostal2Etablissement  | Code_Postal  | Si Type == 'secondaire' |
| codeCommuneEtablissement  | Code_Commune | Si Type == 'principale' |
| codeCommune2Etablissement | Code_Commune | Si Type == 'secondaire' |
|                           |              |                         |
|                           |              |                         |
|                           |              |                         |

In [4]:
insee = AWS_connection.url_instance_bucket(path_file = 'INSEE/Stock/ETS/StockEtablissement_utf8.csv')
ets = AWS_connection.url_instance_bucket(path_file = 'INPI/TC_1/Stock_processed/initial_ETS.gz')
ets

'https://calfdata.s3.eu-west-3.amazonaws.com/INPI/TC_1/Stock_processed/initial_ETS.gz'

## Matching établissement principal

Ici, on filtre les variables communes pour l'INSEE & INPI établissements secondaires.

In [5]:
# load data into dataframes
data_insee_= dd.read_csv(insee,
                         usecols = ['siren',
                                  'siret',
                                 'codePostalEtablissement',
                                 #'codePostal2Etablissement',
                                 'codeCommuneEtablissement',
                                 #'codeCommune2Etablissement'
                                   ],
                       dtype={'siren':'object',
                              'siret':'object',
                              'codePostalEtablissement':'object',
                              #'codePostal2Etablissement':'object',
                              'codeCommuneEtablissement':'object',
                              #'codeCommune2Etablissement':'object',
                             }
                      )

data_ets_= (dd.read_csv(ets, 
                       usecols = [
                           'Type',
                           'Siren',
                                  'Code_Postal',
                                  'Code_Commune'
                                 ],
                       dtype={
                           'Type':'object',
                           'Siren':'object',
                              'Code_Postal':'object',
                              'Code_Commune':'object'
                             },
                      compression='gzip',
                      blocksize=None,
                      low_memory=False
                      )
            .compute()
            .loc[lambda x: x['Type'].isin(['PRI'])]
           )

In [6]:
data_ets_['Type'].unique()

array(['PRI'], dtype=object)

In [7]:
data_ets_.shape

(3017128, 4)

In [10]:
data_ets_.head()

,Siren,Type,Code_Postal,Code_Commune
0,813543063,PRI,01500,01007
16,442377040,PRI,01310,NaN
27,809861297,PRI,01140,01348
32,480952712,PRI,01190,01057
35,388239667,PRI,01140,01420


In [9]:
data_merged = data_insee_.merge(
    data_ets_,
    how= 'right',
    left_on = ["siren", 'codePostalEtablissement'],
    right_on = ['Siren', 'Code_Postal'],
    indicator = True).compute()

UnpicklingError: invalid load key, '\xdd'.